# Lab 2 - Podział danych

Podział danych to jedna z kluczowych czynności podczas przygotowywania ich do wykorzystania wspólnie z algorytmami uczenia maszynowego. Zwykle podział dokonywany jest na trzy rozłączne podzbiory: trenignowy, walidacyjny i testowy.

Podzbiór treningowy stanowi zazwyczaj 80-90% oryginalnego zbioru danych i przeznaczony jest do trenowania modeli predykcyjnych. Pozostałe 10-20% oryginalnego zbioru danych stanowi podzbiór testowy, który służy do końcowej oceny jakości wytrenowanego modelu. Dodatkową ocenę jakości modelu można przeprowadzić za pomocą podzbioru walidacyjnego, który stanowi zwykle 10-20% podzbioru treningowego. Przeznaczeniem podzbioru walidacyjnego jest cykliczna ocena generalizacji wyników modelu predykcyjnego poprzez dostosowanie jego parametrów i hiperparametrów do danych zawartych w tym podzbiorze.

Można z łatwością dokonać manualnej implementacji funkcji/metody odpowiedzialnej za dokonanie podziału, lecz znacznie lepszą drogą będzie wykorzystanie dostępnych (i przetestowanych) implementacji dostarczanych za pomocą bibliotek.

## Scikit-learn

Biblioteka [scikit-learn](https://pypi.org/project/scikit-learn/) jest jednym z najczęściej stosowanych pakietów przez badaczy danych. Scikit-learn nie wchodzi w skład biblioteki standardowej, zatem trzeba ją doinstalować manualnie. W przypadku korzystania ze środowisk Colab lub Conda, implementacja zwykle jest już zainstalowana automatycznie w każdym nowym środowisku roboczym.

Biblioteka scikit-learn - oprócz implementacji funkcji służących do podziału danych - dostarcza także metody przetwarzania i uzupełniania informacji, a także implementacje popularnych algorytmów uczenia maszynowego. Ciekawą funkcjonalnością biblioteki scikit-learn jest zautomatyzowane udostępnianie popularnych (rzeczywistych) zbiorów danych bez konieczności ich manualnego pobierania i rozpakowywania. Pełna lista dostępna jest pod adresem: [https://scikit-learn.org/stable/datasets.html](https://scikit-learn.org/stable/datasets.html).

## Pobieranie danych za pomocą biblioteki scikit-learn

Do pobierania zbiorów danych przeznaczone są funkcje sprecyzowane w podstronach poświęconych każdemu ze zbiorów. Przykładowo, dla zbioru California Housing ([https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)) przeznaczona jest funkcja fetch_california_housing dostępna w module *datasets* pakietu *sklearn*.

Domyślnie zwracany jest słownik zawierający tablice wartości będące atrybutami. Przekazując do funkcji parametr *as_frame* o wartości logicznej *True*, zostanie zwrócony zbiór danych w postaci ramki danych *pandas* dostępne w kluczu *frame* wynikowego słownika.

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
data = fetch_california_housing(as_frame=True)['frame']

In [ ]:
data

## Podział danych

### Podział prosty

Do dokonania podziału prostego przeznaczona jest funkcja *train_test_split* znajdująca się w module *model_selection* pakietu *sklearn*. Najważniejsze parametry przyjmowane przez funkcję to:
- (nienazwane) tablice NumPy z atrybutami warunkowymi oraz opcjonalnie z atrybutem decyzyjnym,
- test_size: odsetek stanowiący rozmiar danych testowych,
- random_state: ziarno losowości.

Wynikiem funkcji będą dwie (dla jednej przekazanej tablicy wejściowej z danymi) lub cztery (dla dwóch tablic wejściowych) tablice stanowiące podział danych wejściowych na dane treningowe i testowe.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

Podział prosty atrybutów warunkowych można przeprowadzić przekazując tylko jedną tablicę wejściową. Podział zostanie przeprowadzony w stosunku 80%:20% odpowiednio dla podzbioru treningowego i testowego.

In [ ]:
X = np.arange(80).reshape((20, 4))  # tablica zawierajace atrybuty warunkowe
# y = range(5)  # tablica zawierajaca jeden atrybut decyzyjny
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [ ]:
len(X), len(X_train), len(X_test)
assert len(X) == len(X_train) + len(X_test)

Dla atrybutów warunkowych i atrybutu decyzyjnego podział ten można przeprowadzić analogicznie.

In [ ]:
X = np.arange(80).reshape((20, 4))  # tablica zawierajace atrybuty warunkowe
y = range(20)  # tablica zawierajaca jeden atrybut decyzyjny
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

Warto zauważyć, że istotną wadą podziału prostego jest brak zachowania proporcji względem wybranego atrybutu. Przykładowo, jeżeli w atrybucie decyzyjnym znajdzie się dziesięć zer i dziesięć jedynek, nie mamy kontroli nad tym czy w podzbiorze testowym (o rozmiarze 4) znajdą się same zera, czy dwa zera i dwie jedynki, co jest proporcją zgodną z pełnym oryginalnym zbiorem danym.

### Podział warstwowy

W bibliotece scikit-learn znajduje się klasa *StratifiedShuffleSplit*, której przeznaczeniem jest dokonywanie wielokrotnych podziałów z zachowaniem proporcji występujących w oryginalnej populacji. Inicjalizator klasy *StratifiedShuffleSplit* przyjmuje następujące parametry:
- n_splits: liczba podziałów,
- test_size: odsetek obiektów stanowiących podzbiór testowy,
- random_state: ziarno losowości.

Na obiekcie klasy *StratifiedShuffleSplit* jest dostępna metoda *split*, która przyjmuje parametry w postaci tablicy zawierającej zestaw atrybutów warunkowych oraz tablicy z atrybutem decyzyjnym. Wynikiem funkcji jest iterator, po którym można przeiterować się za pomocą pętli for. Liczba przebiegów będzie zgodna z wartością parametru *n_splits* inicjalizatora klasy *StratifiedShuffleSplit*, a w każdym przebiegu pętli obiekt sterujący będzie stanowiła krota zawierająca dwa elementy: tablicę indeksów obiektów z oryginalnego zbioru danych stanowiącego podzbiór treningowy oraz testowy.


Na potrzeby przykładu wygenerujemy nowy atrybut decyzyjny, który będzie zawierał wartości 0 i 1 w proporcji 50%:50%.

In [ ]:
data_size = len(data)
half_data_size = data_size // 2
data['decision'] = np.array([0] * half_data_size + [1] * half_data_size)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
output_col = 'decision'

for train_idx, test_idx in splitter.split(data.loc[:, data.columns != output_col], data[output_col]):
  print(f'{train_idx}\n{test_idx}\n{len(train_idx)}, {len(test_idx)}\n------------------\n')

## Zadania

1. Przygotować funkcję get_dataset(name: str) -> pd.DataFrame, która zwróci ramkę danych z wczytanym zbiorem danych dostępnym w pakiecie Scikit-learn. Jako nazwę można przyjąć dowolny identyfikator, np. alias w adresie URL prowadzącym do szczegółów zbioru: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset (alias jest dostępny po znaku #).

2. Dokonać podziału podzbioru treningowego w stosunku 80%:20% przeznaczając 20% na podzbiór walidacyjny, gdzie pozostałe 80% nadal będzie stanowiło podzbiór treningowy, lecz okrojony.

3. Dla przeprowadzonych podziałów metodą prostą oraz warstwową (dla każdego podziału) przygotować histogram prezentujący rozkład liczebności wartości atrybutu decyzyjnego zarówno w podzbiorze treningowym, jak i testowym.

4. Dokonać podziału oryginalnego zbioru danych metodą warstwową względem innego atrybutu (obecnego w pierwotnej wersji zbioru). Jakiego atrybutu i dlaczego warto użyć? Dopuszczalne są niewielkie i uzasadnione zmiany wartości atrybutu stanowiącego źródło proporcji podziału.